# Data Cleaning 

## Objectives

* Write here your notebook objective, for example, "Fetch data from Kaggle and save as raw data", or "engineer features for modelling"

## Inputs

* Write here which data or information you need to run the notebook 

## Outputs

* Write here which files, code or artefacts you generate by the end of the notebook 

## Additional Comments

* In case you have any additional comments that don't fit in the previous bullets, please state them here. 


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [3]:
import os
current_dir = os.getcwd()
current_dir

'c:\\project-five-golf-data-analytics\\jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [4]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [5]:
current_dir = os.getcwd()
current_dir

'c:\\project-five-golf-data-analytics'

# Load Data

The initial data to load is the csv file that has oalready been partly cleaned in the TopTenPlayerStudy notebook.

In [6]:
import pandas as pd

file_path = "outputs/data/interim/cleaned_data.csv"
df = pd.read_csv(file_path)
df.head()


,Player_initial_last,tournament id,player id,hole_par,strokes,hole_DKP,hole_FDP,hole_SDP,streak_DKP,streak_FDP,...,sg_app,sg_ott,sg_t2g,sg_total,finish_clean,finish_numeric,pos_differs,true_pos,top_ten,mid_band
0,A. Ancer,401353224,9261,288,289,60.0,51.1,56,3,7.6,...,-0.08,0.86,0.65,0.85,32,32.0,False,32.0,0,0
1,A. Hadwin,401353224,5548,288,286,72.5,61.5,61,8,13.0,...,0.31,0.18,1.24,1.60,18,18.0,False,18.0,0,1
2,A. Lahiri,401353224,4989,144,147,21.5,17.4,27,0,0.0,...,-1.09,0.37,0.02,-0.54,CU,0.0,False,0.0,0,0
3,A. Long,401353224,6015,144,151,20.5,13.6,17,0,0.4,...,-0.02,0.80,-1.08,-2.54,CU,0.0,False,0.0,0,0
4,A. Noren,401353224,3832,144,148,23.5,18.1,23,0,1.2,...,-1.39,0.19,-1.56,-1.04,CU,0.0,False,0.0,0,0


---

# Remove unecessary features

A number of features in this df are unecessary for the client's needs. We will list the features, highlight the ones needed and delete the rest.

In [7]:
df.columns.tolist()

['Player_initial_last',
 'tournament id',
 'player id',
 'hole_par',
 'strokes',
 'hole_DKP',
 'hole_FDP',
 'hole_SDP',
 'streak_DKP',
 'streak_FDP',
 'streak_SDP',
 'n_rounds',
 'made_cut',
 'pos',
 'finish_DKP',
 'finish_FDP',
 'finish_SDP',
 'total_DKP',
 'total_FDP',
 'total_SDP',
 'player',
 'Unnamed: 2',
 'Unnamed: 3',
 'Unnamed: 4',
 'tournament name',
 'course',
 'date',
 'purse',
 'season',
 'no_cut',
 'Finish',
 'sg_putt',
 'sg_arg',
 'sg_app',
 'sg_ott',
 'sg_t2g',
 'sg_total',
 'finish_clean',
 'finish_numeric',
 'pos_differs',
 'true_pos',
 'top_ten',
 'mid_band']

In [8]:
features_to_keep = [
    "player id",
    "tournament id",
    "finish_numeric",
    "true_pos",
    "top_ten",
    "mid_band",
    "sg_putt",
    "sg_arg",
    "sg_app",
    "sg_ott",
    "sg_t2g",
    "sg_total"
]

df = df[features_to_keep]
df.head()


,player id,tournament id,finish_numeric,true_pos,top_ten,mid_band,sg_putt,sg_arg,sg_app,sg_ott,sg_t2g,sg_total
0,9261,401353224,32.0,32.0,0,0,0.20,-0.13,-0.08,0.86,0.65,0.85
1,5548,401353224,18.0,18.0,0,1,0.36,0.75,0.31,0.18,1.24,1.60
2,4989,401353224,0.0,0.0,0,0,-0.56,0.74,-1.09,0.37,0.02,-0.54
3,6015,401353224,0.0,0.0,0,0,-1.46,-1.86,-0.02,0.80,-1.08,-2.54
4,3832,401353224,0.0,0.0,0,0,0.53,-0.36,-1.39,0.19,-1.56,-1.04


## Investigate missing data

In [16]:
num_rows = df.shape[0]
print(num_rows)

df.isnull().sum()



36864


player id            0
tournament id        0
finish_numeric       0
true_pos             0
top_ten              0
mid_band             0
sg_putt           7684
sg_arg            7684
sg_app            7684
sg_ott            7684
sg_t2g            7684
sg_total          7683
dtype: int64

A propoortion of the entries are rows are missing strokes gained data. Firstly, we will check to see if these are proportionally evenly split between top tens and non-top tens.

In [22]:
complete_sg = df.dropna(subset=['sg_putt','sg_arg','sg_app','sg_ott','sg_t2g','sg_total'])

top_ten_counts = complete_sg['top_ten'].value_counts()
top_ten_proportions = complete_sg['top_ten'].value_counts(normalize=True)

top_ten_counts, top_ten_proportions



(top_ten
 0    26367
 1     2813
 Name: count, dtype: int64,
 top_ten
 0    0.903598
 1    0.096402
 Name: proportion, dtype: float64)

In [ ]:
missing_sg = df[df[['sg_putt','sg_arg','sg_app','sg_ott','sg_t2g','sg_total']].isnull().any(axis=1)]

top_ten_counts_missing = missing_sg['top_ten'].value_counts()
top_ten_proportions_missing = missing_sg['top_ten'].value_counts(normalize=True)

top_ten_counts_missing, top_ten_proportions_missing


(top_ten
 0    6749
 1     935
 Name: count, dtype: int64,
 top_ten
 0    0.878319
 1    0.121681
 Name: proportion, dtype: float64)

Given that the proportion of top-ten to non-top-ten players in the rows with missing strokes gained data is broadly similar to that in the overall dataset, and considering that imputing these missing values would be challenging (for example, assigning the mean strokes gained to top or bottom finishers would not accurately reflect their performance) and could distort performance metrics, the most appropriate approach is to remove rows with missing strokes gained statistics

In [25]:
df = df.dropna(subset=['sg_putt','sg_arg','sg_app','sg_ott','sg_t2g','sg_total'])

num_rows = df.shape[0]
print(num_rows)


29180


## rename features of consistency

In [31]:
df.rename(columns=lambda x: x.strip().lower().replace(' ', '_'), inplace=True)

df.head()


,player_id,tournament_id,finish_numeric,true_pos,top_ten,mid_band,sg_putt,sg_arg,sg_app,sg_ott,sg_t2g,sg_total
0,9261,401353224,32.0,32.0,0,0,0.20,-0.13,-0.08,0.86,0.65,0.85
1,5548,401353224,18.0,18.0,0,1,0.36,0.75,0.31,0.18,1.24,1.60
2,4989,401353224,0.0,0.0,0,0,-0.56,0.74,-1.09,0.37,0.02,-0.54
3,6015,401353224,0.0,0.0,0,0,-1.46,-1.86,-0.02,0.80,-1.08,-2.54
4,3832,401353224,0.0,0.0,0,0,0.53,-0.36,-1.39,0.19,-1.56,-1.04


## Check for other unusual data entries.

In [35]:
placeholders = ['n/a', 'na', '-', 'none', 'null', 'unknown', 'missing', '']

for col in df.columns:
    df[col] = df[col].replace(placeholders, pd.NA)

In [36]:
missing_summary = df.isna().sum().to_frame('missing_count')
missing_summary['missing_percent'] = (missing_summary['missing_count'] / len(df)) * 100
missing_summary = missing_summary[missing_summary['missing_count'] > 0].sort_values('missing_percent', ascending=False)
print(missing_summary)

Empty DataFrame
Columns: [missing_count, missing_percent]
Index: []


## Check for outliers in strokes gained features


In [37]:
sg_cols = ['sg_putt', 'sg_arg', 'sg_app', 'sg_ott', 'sg_t2g', 'sg_total']
df[sg_cols].describe().T


,count,mean,std,min,25%,50%,75%,max
sg_putt,29180.0,-0.121005,1.119451,-5.99,-0.77,-0.04,0.63,4.43
sg_arg,29180.0,-0.040744,0.727315,-6.43,-0.45,0.00,0.42,3.17
sg_app,29180.0,-0.101759,1.119763,-9.25,-0.74,0.00,0.64,4.67
sg_ott,29180.0,-0.045896,0.808139,-7.74,-0.45,0.05,0.48,2.77
sg_t2g,29180.0,-0.188346,1.640817,-13.95,-1.08,-0.01,0.92,6.30
sg_total,29180.0,-0.305578,1.966647,-13.67,-1.37,-0.16,1.06,8.52


## Clean data for players who missed the cut.
Curretly, in the 'true_pos' feature, players who missed the cut have been given the numeric value of 0. This is going to cause issues with a regression model as it will appear that these players performed better than those with a position than 1 (tournament winner). Analysis needs to be done to ascertain what to do with these values.

In [39]:
df['true_pos'].describe()

count    29180.000000
mean        19.204010
std         25.979197
min          0.000000
25%          0.000000
50%          6.000000
75%         36.000000
max        999.000000
Name: true_pos, dtype: float64

A max value of 999 seems unlikely!

In [40]:
high_positions = df[df['true_pos'] > 100]

print(high_positions)


       player_id  tournament_id  finish_numeric  true_pos  top_ten  mid_band  \
25443       7081           2706             0.0     999.0        0         0   
31177       3950           2514             0.0     999.0        0         0   
31693        512           2484             0.0     999.0        0         0   
33998       1411           2250             0.0     999.0        0         0   

       sg_putt  sg_arg  sg_app  sg_ott  sg_t2g  sg_total  
25443    -0.75    1.43   -0.54   -0.83    0.07     -0.68  
31177     1.07   -0.41    0.36    0.87    0.81      1.89  
31693     0.93    0.19    1.06   -0.03    1.23      2.16  
33998     1.11    0.17   -0.55   -1.32   -1.70     -0.59  


As this is only 4 occurences we can delete them.

In [41]:
df = df[df['true_pos'] != 999.0]
df['true_pos'].describe()


count    29176.000000
mean        19.069681
std         23.310376
min          0.000000
25%          0.000000
50%          6.000000
75%         36.000000
max         91.000000
Name: true_pos, dtype: float64

This is now more realistic. In the majority of golf tournaments, the top 70 players make the cut out of a field of around 120. Therefore, changing the players who have a missed cut to a value of 91 (the current max finish) seems realistic.

In [42]:
df.loc[df['true_pos'] == 0, 'true_pos'] = 91
print(df[df['true_pos'] == 91])

       player_id  tournament_id  finish_numeric  true_pos  top_ten  mid_band  \
2           4989      401353224             0.0      91.0        0         0   
3           6015      401353224             0.0      91.0        0         0   
4           3832      401353224             0.0      91.0        0         0   
5           5502      401353224             0.0      91.0        0         0   
9           9484      401353224             0.0      91.0        0         0   
...          ...            ...             ...       ...      ...       ...   
36737       4643           2260             0.0      91.0        0         0   
36741        215           2260             0.0      91.0        0         0   
36743       3295           2260             0.0      91.0        0         0   
36744       1157           2260             0.0      91.0        0         0   
36752       4708           2260             0.0      91.0        0         0   

       sg_putt  sg_arg  sg_app  sg_ott 

In [43]:
df['true_pos'].describe()

count    29176.000000
mean        59.170688
std         32.481768
min          1.000000
25%         29.000000
50%         64.000000
75%         91.000000
max         91.000000
Name: true_pos, dtype: float64

## Push files to Repo

In [44]:
output_dir = 'outputs/data/final'
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, 'cleaned_golfdata.csv')
df.to_csv(output_path, index=False)
print(f"Saved cleaned data to {output_path}")


Saved cleaned data to outputs/data/final\cleaned_golfdata.csv
